In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

In [7]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information",
    backstory="""
    You produce the highest quaility research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter = 10,
    verbose = True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose = True,
)

In [11]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output = "Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be",
    human_input = True,
    output_file="./files/youtube_ai/brainstorm_task.md",
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input = True,
    context=[
        brainstorm_task,
    ],
    output_file="./files/youtube_ai/inspiration_task.md"
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="Your answer must have all the information a writer would need to write a Youtube script.",
    async_execution = True,
    context=[
        selection_task,
    ],
    output_file="./files/youtube_ai/research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible",
    agent=researcher,
    async_execution = True,
    context=[
        selection_task,
    ],
    output_file="./files/youtube_ai/competitors_task.md"
)

inspiration_task = Task(
    description="Search for vidoes or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution = True,
    context=[
        selection_task,
    ],
    output_file="./files/youtube_ai/inspiration_task.md"
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        research_task,
        selection_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="./files/youtube_ai/script_task.md"
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

result

 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry


> Entering new CrewAgentExecutor chain...
Thought: To come up with compelling and viral video ideas for a YouTube channel in the hot sauce industry, I need to understand current trends, popular content styles, and what appeals to a young audience. I'll start by asking the Senior Researcher for information on current trends and popular content in the food and hot sauce industries on YouTube.

Action: Ask question to co-worker
Action Input: {"co-worker": "Senior Researcher", "question": "Can you provide me with current trends and popular content styles in the food and hot sauce industries on YouTube? I need to understand what is trending and what kind of content is engaging a young audience.", "context": "I am coming up with 5 video ideas for a YouTube channel in the hot sauce industry, targeting a young audience. The content needs to be vira

'### Script for "Extreme Hot Sauce Challenge" Video:\n\n---\n\n#### **Title**:\n"Extreme Hot Sauce Challenge: Can We Handle the Heat?"\n\n#### **Thumbnail Prompt**:\nCreate a thumbnail featuring Alex and Jamie with exaggerated facial expressions, holding hot wings, and with flames in the background. Include a text overlay that says "Extreme Hot Sauce Challenge!" and images of hot sauce bottles with a heat meter.\n\n---\n\n#### **Introduction**:\n\n**[Opening Scene: Upbeat music plays in the background. The hosts, Alex and Jamie, stand in front of a table filled with various hot sauces. They are excited and energetic.]**\n\n**Alex**: "Hey everyone! Welcome back to our channel! I\'m Alex."\n\n**Jamie**: "And I\'m Jamie. Today, we have a fiery challenge lined up for you - the Extreme Hot Sauce Challenge!"\n\n**Alex**: "That\'s right! We\'ll be trying out some of the spiciest hot sauces on the planet, starting from mild and working our way up to the extreme heat."\n\n**Jamie**: "But that\'